**Konfiguracja środowiska i łączenie z dyskiem Google**

Komórkę poniżej należy odpalić przy każdym nowym uruchomieniu notebooka.

https://stackoverflow.com/questions/52681405/how-can-i-import-custom-modules-from-a-github-repository-in-google-colab

https://medium.com/analytics-vidhya/importing-your-own-python-module-or-python-file-into-colab-3e365f0a35ec



In [1]:
!pip install PyPDF2
!pip install Jinja2
!pip install num2words
!sudo apt-get install tex-common texlive-base texlive-binaries texlive-fonts-extra texlive-latex-extra texlive-lang-polish texlive-luatex
!sudo apt install dvipng
#!pdflatex -help

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import io
import glob
import shutil

import datetime
import numpy as np
import pandas as pd

import PyPDF2
import jinja2
from jinja2 import Template
from num2words import num2words


%cd /content/drive/MyDrive/KL\ Nord/
import KLutils
!ls

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tex-common is already the newest version (6.13).
texlive-base is already the newest version (2019.20200218-1).
texlive-binaries is already the newest version (2019.20190605.51237-3build2).
texlive-fonts-extra is already the newest version (2019.202000218-1).
texlive-lang-polish is already the newest version (2019.20200218-1).
texlive-latex-extra is already the newest version (2019.202000218-1).
texlive-luatex is already the newest version (2019.20200218-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state infor

**Wybranie pliku z komputera**

Należy za każdym razem wybrać plik

In [9]:
# Choose file to process
# assuming only one file chosen
uploaded   = files.upload()
key        = list( uploaded.keys() )[0]
pdfFileObj = io.BytesIO( uploaded[key] )

template_file = 'KL_invoice_template.tex'

Saving 95262763_inv._Unip..PDF to 95262763_inv._Unip..PDF


**Wybierz plik .pdf ze swojego komputera w oknie powyżej.**

Pobrany plik powinien się pojawić w tym samym folderze co ten notebook na Dysku Google.

**Wygeneruj fakturę od KL Nord**

Uzupełnij numer faktury (KL Nord)

In [14]:
klno = '2-2023.KL_Nord'    # TODO: Change filename / invoce id          # <----------------------------------- Tutaj zmienić numer!
eur_per_t = None           # put value manually if needed, otherwise put None

benef_name = { 'Symetal' : r'\textbf{SYMETAL Aluminiu Foil Industry S.A.}',
                 'Elval' : r'\textbf{ELVALHALCOR Hellenic Copper and Aluminium Industry S.A.}' }
benef_vatno = {'Symetal' : 'EL 998104946',
                 'Elval' : 'EL 094061318'  }




# ============================================== CODE GENERATING INVOICE ==========================================================

inv_no, date, mass, cost, client, benef = KLutils.pdf_to_data(pdfFileObj)

if eur_per_t is None:
    eur_per_t = { 'Elval'   : 40.0,
                  'Symetal' : 60.0 }
    eur_per_t = eur_per_t[benef]
value = mass * eur_per_t / 1000

valuesayen, valuesaypl = KLutils.valuesay(value)


print('client:     ', client )
print('beneficient:', benef )
print('mass:       ', mass )
print()

# save data to excel
data = { 'klno'       : klno,
         'invno'      : inv_no,
         'date'       : date,
         'bname'      : benef_name[benef],
         'vatno'      : benef_vatno[benef],
         'mass'       : mass/1000,
         'value'      : value,
         'valuesaypl' : valuesaypl,
         'valuesayen' : valuesayen,
         'eurpert'    : eur_per_t }


template = ''
with open(template_file, 'r') as f:
    template = f.read()

j2_template = Template(template)
#print(j2_template.render(data))

with open('{}.tex'.format(klno), "w") as tex_file:
    tex_file.write(j2_template.render(data))

os.system('lualatex -interaction=nonstopmode {}.tex'.format(klno) )
print()

# remove unnecessary temporary files
os.system( 'rm {}.tex'.format(klno) )
os.system( 'rm {}.log'.format(klno) )
os.system( 'rm {}.out'.format(klno) )
os.system( 'rm {}.aux'.format(klno) )

client:      PRZEDSIEBIORSTWO PRODUKCYJNO HANDLOUSLUGOWE UNIPACO SPOLKA AKCYJNA
beneficient: Symetal
mass:        15170




0


# Sources

http://eosrei.net/articles/2015/11/latex-templates-python-and-jinja2-generate-pdfs

https://github.com/savoirfairelinux/num2words


In [4]:
print( KLutils.pdf_to_data(pdfFileObj) )

(95261390, datetime.date(2023, 1, 18), 10376, 61208.03, 'IGLOOCAR Sp. z o.o.Al. Jana Pawla II 11', 'Elval')


In [13]:
valuesayen, valuesaypl = KLutils.valuesay(value)
print(valuesayen)
print(valuesaypl)

nine hundred and ten point two
dziewi\k{e}{'c}set dziesi\k{e}{'c} przecinek dwa


In [7]:
num2words(61208.03)

'sixty-one thousand, two hundred and eight point zero three'